In [1]:
import os
from typing import Optional
from components.company_report import CompanyReport

from components.data_disambiguation import DataDisambiguation
from components.question_proposal_generator import (
    QuestionProposalGenerator,
)
from components.summarize_cypher_result import SummarizeCypherResult
from components.text2cypher import Text2Cypher
from components.unstructured_data_extractor import (
    DataExtractor,
    DataExtractorWithSchema,
)
from driver.neo4j import Neo4jDatabase
from fastapi import FastAPI, HTTPException, WebSocket, WebSocketDisconnect
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
from fewshot_examples import get_fewshot_examples
from llm.llama import LlamaChat
from pydantic import BaseModel

In [2]:
HARD_LIMIT_CONTEXT_RECORDS = 10

neo4j_connection = Neo4jDatabase(
    host=os.environ.get("NEO4J_URL", "neo4j+s://demo.neo4jlabs.com"),
    user=os.environ.get("NEO4J_USER", "companies"),
    password=os.environ.get("NEO4J_PASS", "companies"),
    database=os.environ.get("NEO4J_DATABASE", "companies"),
)


  This is the schema representation of the Neo4j database.
  Node properties are the following:
  [{'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}, {'property': 'summary', 'type': 'STRING'}], 'labels': 'Person'}, {'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'nbrEmployees', 'type': 'INTEGER'}, {'property': 'isDissolved', 'type': 'BOOLEAN'}, {'property': 'id', 'type': 'STRING'}, {'property': 'motto', 'type': 'STRING'}, {'property': 'summary', 'type': 'STRING'}, {'property': 'isPublic', 'type': 'BOOLEAN'}, {'property': 'revenue', 'type': 'FLOAT'}], 'labels': 'Organization'}, {'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}], 'labels': 'IndustryCategory'}, {'properties': [{'property': 'id', 'type': 'STRING'}, {'property': 'summary', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}], 'labels': 'City'}, {'properties': [{'property': 'name', 'type': 'STRING'}, {'property':

In [3]:
default_llm = LlamaChat()
summarize_results = SummarizeCypherResult(
    llm=LlamaChat(
        max_tokens=128,
    )
)

getting the model


Done!!!!!!
getting the model
Done!!!!!!


In [4]:
text2cypher = Text2Cypher(
                database=neo4j_connection,
                llm=default_llm,
                cypher_examples=get_fewshot_examples(),
            )

In [5]:
question = {"type": "question","data": "what is the dataset about"}

In [11]:
# question = data["question"]
chatHistory = []
chatHistory.append({"role": "user", "content": question})
print(f"received question: {question}")
results = None
try:
    results = text2cypher.run(question, chatHistory)
    print("results", results)
except Exception as e:
    print(str(e))
    # continue
if results is None:
    print("Could not generate Cypher statement")
    # continue


output = summarize_results.run_async(
    question,
    results["output"][:HARD_LIMIT_CONTEXT_RECORDS],
)
chatHistory.append({"role": "system", "content": output})

received question: {'type': 'question', 'data': 'what is the dataset about'}
can only concatenate str (not "dict") to str
Could not generate Cypher statement


TypeError: 'NoneType' object is not subscriptable

In [13]:
results

In [10]:
onToken

NameError: name 'onToken' is not defined